# Catalyst Cooperative Jupyter Notebook Template
This notebook lays out a standard format and some best practices for creating interactive / exploratory notebooks which can be relatively easily shared between different PUDL users, and turned into reusable Python modules for integration into our underlying Python packages.

## Begin with a narrative outline
Each notebook should start with a brief explanation (in Markdown) explaining the purpose of the analysis, and outlining the different stages / steps which are taken to accomplish the analysis.
As the analysis develops, you can add new sections or details to this section.

## Notebooks should be runnable
Insofar as it's possible, another PUDL user who has cloned the repository that the notebook is part of should be able to update their `pudl-dev` conda environment, open the notebook, and run all cells successfully.
If there are required data or other prerequisites that the notebook cannot manage on its own -- like a file that needs to be downloaded by hand and placed in a particular location -- those steps should be laid out clearly at the beginning of the notebook.

## Avoid Troublesome Elements

### Don't hardcode passwords or access tokens
Most of our work is done in public Github repositories.
No authentication information should ever appear in a notebook.
These values can be stored in environment variables on your local computer.

### Do not hardocde values, especially late in the notebook
If the analysis depends on particular choices of input values, those should be called out explicitly at the beginning of the notebook.
(NB: We should explore ways to parameterize notebooks, [papermill](https://papermill.readthedocs.io/en/latest/) is one tool that does this).

### Don't hardcode absolute file paths
If anyone is going to be able to use the notebook, the files it uses will need to be stored somewhere that makes sense on both your and other computers.
We assume that anyone using this template has the PUDL package installed, and has a local PUDL data management environment set up.
  * Input data that needs to be stored on disk and accessed via a shared location should be saved under `<PUDL_IN>/data/local/<data_source>/`.
  * Intermediate saved data products (e.g. a pickled result of a computationally intensive process) and results should be saved to a location relative to the notebook, and within the directory hierarchy of the repository that the notebook is part of.
  
### Don't require avoidable long-running computations
Consider persisting to disk the results of computations that take more than a few minutes, if the outputs are small enough to be checked into the repository and shared with other users.
Only require the expensive computation to be run if this pre-computed output is not available.

### Don't litter
Don't leave lots of additional code laying around, even commented out, "just in case" you want to look at it again later.
Notebooks need to be relatively linear in the end, even though the thought processes and exploratory analyses that generate them may not be.
Once you have a working analysis, either prune those branches, or encapsulate them as options within functions.

### Don't load unneccesary libraries
Only import libraries which are required by the notebook, to avoid unnecessary dependencies.
If your analysis requires a new library that isn't yet part of the shared `pudl-dev` environment, add it to the `devtools/environment.yml` file so that others will get it when they update their environment.

## Related Resources:
Lots of these guidelines are taken directly from Emily Riederer's post: [RMarkdown Driven Development](https://emilyriederer.netlify.app/post/rmarkdown-driven-development/).
For more in depth explanation of the motivations behind this layout, do go check it out!

# Import Libraries
* Because it's very likely that you will be editing the PUDL Python packages or your own local module under development while working in the notebook, use the %autoreload magic with autoreload level 2 to ensure that any changes you've made in those files are always reflected in the code that's running in the notebook.
* Put all import statements at the top of the notebook, so everyone can see what its dependencies are up front, and so that if an import is going to fail, it fails early, before the rest of the notebook is run.
* Try to avoid importing individual functions and classes from deep within packages, as it may not be clear to other users where those elements came from, later in the notebook, and also to minimize the chance of namespace collisions.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard libraries
import logging
import os
import pathlib
import sys

# 3rd party libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sa

# Local libraries
import pudl

# Configure Display Parameters

In [3]:
sns.set()
%matplotlib inline
mpl.rcParams['figure.figsize'] = (10,4)
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# Use Python Logging facilities
* Using a logger from the beginning will make the transition into the PUDL package easier.
* Creating a logging handler here will also allow you to see the logging output coming from PUDL and other underlying packages.

In [4]:
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

# Define Functions
In many cases, the eventual product of a notebook analysis is going to be the creation of new, reusable functions that are integrated into the underlying PUDL code. You should begin the process of accumulating and organizing those functions as soon as you notice repeated patterns in your code.
* Functions should be used to encapsulate any potentially reusable code.
* Functions should be defined immediately after the imports, to avoid accidental dependence on zombie variables that are defined further down in the code
* While they may evolve over time, having brief docstrings explaining what they do will help others understand them.
* If there's a particular type of plot or visualization that is made repeatedly in the notebook, it's good to parameterize and functionalize it here too, so that as you refine the presentation of the data and results, those improvements can be made in a single place, and shown uniformly throughout the notebook.
* As these functions mature and become more general purpose tools, you will probably want to start migrating them into their own local module, under a `src` directory in the same directory as the notebook. You will want to import this module 

## Dummy EIA 861 ETL

In [5]:
def test_etl_eia(eia_inputs, pudl_settings):
    """
    This is a dummy function that runs the first part of the EIA ETL
    process -- everything up until the entity harvesting begins. For
    use in this notebook only.

    """
    eia860_tables = eia_inputs["eia860_tables"]
    eia860_years = eia_inputs["eia860_years"]
    eia861_tables = eia_inputs["eia861_tables"]
    eia861_years = eia_inputs["eia861_years"]
    eia923_tables = eia_inputs["eia923_tables"]
    eia923_years = eia_inputs["eia923_years"]

    # generate CSVs for the static EIA tables, return the list of tables
    #static_tables = _load_static_tables_eia(datapkg_dir)

    # Extract EIA forms 923, 860
    eia860_raw_dfs = pudl.extract.eia860.Extractor().extract(eia860_years, testing=True)
    eia861_raw_dfs = pudl.extract.eia861.Extractor().extract(eia861_years, testing=True)
    eia923_raw_dfs = pudl.extract.eia923.Extractor().extract(eia923_years, testing=True)

    # Transform EIA forms 860, 861, 923
    eia860_transformed_dfs = pudl.transform.eia860.transform(eia860_raw_dfs, eia860_tables=eia860_tables)
    eia861_transformed_dfs = pudl.transform.eia861.transform(eia861_raw_dfs, eia861_tables=eia861_tables)
    eia923_transformed_dfs = pudl.transform.eia923.transform(eia923_raw_dfs, eia923_tables=eia923_tables)

    # create an eia transformed dfs dictionary
    eia_transformed_dfs = eia860_transformed_dfs.copy()
    eia_transformed_dfs.update(eia861_transformed_dfs.copy())
    eia_transformed_dfs.update(eia923_transformed_dfs.copy())

    # convert types..
    eia_transformed_dfs = pudl.helpers.convert_dfs_dict_dtypes(eia_transformed_dfs, 'eia')

    return eia860_raw_dfs, eia861_raw_dfs, eia923_raw_dfs, eia_transformed_dfs

# Define Notebook Parameters
If there are overarching parameters which determine the nature of the analysis -- which US states to look at, which utilities are of interest, a particular start and end date -- state those clearly at the beginning of the analysis, so that they can be referred to by the rest of the notebook and easily changed if need be.
* If the analysis depends on local (non-PUDL managed) datasets, define the paths to those data here.
* If there are external URLs or other resource locations that will be accessed, define those here as well.
* This is also where you should create your `pudl_settings` dictionary and connections to your local PUDL databases

In [6]:
EIA861_YEARS = list(range(2001, 2019))
pudl_settings = pudl.workspace.setup.get_defaults()
display(pudl_settings)

ferc1_engine = sa.create_engine(pudl_settings['ferc1_db'])
display(ferc1_engine)

pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
display(pudl_engine)

# Is there other external data you need to pull in?
# If so, put it in a (relatively) standard place on the filesystem.
my_new_data_url = "https://mynewdata.website.gov/path/to/new/data.csv"
my_new_datadir = pathlib.Path(pudl_settings["data_dir"]) / "local/new_data_source"

# Store API keys and other secrets in environment variables
# and read them in here, if needed:
# API_KEY_EIA = os.environ["API_KEY_EIA "]
# API_KEY_FRED = os.environ["API_KEY_FRED "]

{'pudl_in': '/home/zane/code/catalyst/pudl-work',
 'data_dir': '/home/zane/code/catalyst/pudl-work/data',
 'settings_dir': '/home/zane/code/catalyst/pudl-work/settings',
 'pudl_out': '/home/zane/code/catalyst/pudl-work',
 'sqlite_dir': '/home/zane/code/catalyst/pudl-work/sqlite',
 'parquet_dir': '/home/zane/code/catalyst/pudl-work/parquet',
 'datapkg_dir': '/home/zane/code/catalyst/pudl-work/datapkg',
 'notebook_dir': '/home/zane/code/catalyst/pudl-work/notebook',
 'ferc1_db': 'sqlite:////home/zane/code/catalyst/pudl-work/sqlite/ferc1.sqlite',
 'pudl_db': 'sqlite:////home/zane/code/catalyst/pudl-work/sqlite/pudl.sqlite'}

Engine(sqlite:////home/zane/code/catalyst/pudl-work/sqlite/ferc1.sqlite)

Engine(sqlite:////home/zane/code/catalyst/pudl-work/sqlite/pudl.sqlite)

# Load Data
* Given the above parameters and functions, it should now be possible to load the required data into local variables for further wrangling, analysis, and visualization.
* If the data is not yet present on the machine of the person running the notebook, this step should also acquire the data from its original source, and place it in the appropriate location under `<PUDL_IN>/data/local/`.
* If there are steps which have to be done manually here, put them first so that they fail first if the user hasn't read the instructions, and they can fix the situation before a bunch of other work gets done. Try to minimize the amount of work in the filesystem that has to be done manually though.
* If this process takes a little while, don't be shy about producing `logging` output.
* Using the `%%time` cell magic can also help users understand which pieces of work / data acquisition are hard:

## EIA 861 (2010-2018)
* Not yet fully integrated into PUDL
* Post-transform harvesting process isn't compatible w/ EIA 861 structure
* Only getting the `sales_eia861`, `balancing_authority_eia861`, and `service_territory_eia861` tables

### Already Transformed EIA 861 DataFrames

In [7]:
%%time
eia_inputs = {
    "eia860_years": [],
    "eia860_tables": pudl.constants.pudl_tables["eia860"],
    "eia861_years": EIA861_YEARS,
    "eia861_tables": pudl.constants.pudl_tables["eia861"],
    "eia923_years": [],
    "eia923_tables": pudl.constants.pudl_tables["eia923"],
}
eia860_raw_dfs, eia861_raw_dfs, eia923_raw_dfs, eia_transformed_dfs = test_etl_eia(eia_inputs=eia_inputs, pudl_settings=pudl_settings)

No years given. Not extracting eia860 spreadsheet data.
No page for eia861 advanced_metering_infrastructure_eia861 2001
No page for eia861 advanced_metering_infrastructure_eia861 2002
No page for eia861 advanced_metering_infrastructure_eia861 2003
No page for eia861 advanced_metering_infrastructure_eia861 2004
No page for eia861 advanced_metering_infrastructure_eia861 2005
No page for eia861 advanced_metering_infrastructure_eia861 2006
Loading dataframe for eia861 advanced_metering_infrastructure_eia861 2007
Loading dataframe for eia861 advanced_metering_infrastructure_eia861 2008
Loading dataframe for eia861 advanced_metering_infrastructure_eia861 2009
Loading dataframe for eia861 advanced_metering_infrastructure_eia861 2010
Loading dataframe for eia861 advanced_metering_infrastructure_eia861 2011
Loading dataframe for eia861 advanced_metering_infrastructure_eia861 2012
Loading dataframe for eia861 advanced_metering_infrastructure_eia861 2013
Loading dataframe for eia861 advanced_mete

# Sanity Check Data
If there's any validation that can be done on the data which you've loaded to flag if/when it is inappropriate for the analysis that follows, do it here. If you find the data is unusable, use `assert` statements or `raise` Exceptions to stop the notebook from proceeding, and indicate what the problem is.

In [8]:
list(eia861_raw_dfs.keys())

['advanced_metering_infrastructure_eia861',
 'balancing_authority_eia861',
 'demand_response_eia861',
 'demand_side_management_eia861',
 'distributed_generation_eia861',
 'distribution_systems_eia861',
 'dynamic_pricing_eia861',
 'frame_eia861',
 'green_pricing_eia861',
 'mergers_eia861',
 'net_metering_eia861',
 'non_net_metering_eia861',
 'operational_data_eia861',
 'reliability_eia861',
 'sales_eia861',
 'service_territory_eia861',
 'short_form_eia861',
 'utility_data_eia861']

In [9]:
list(eia_transformed_dfs.keys())

['service_territory_eia861',
 'balancing_authority_eia861',
 'sales_eia861',
 'demand_response_eia861']

# Preliminary Data Wrangling
Once all of the data is loaded and looks like it's in good shape, do any initial wrangling that's specific to this particular analysis. This should mostly make use of the higher level functions which were defined above. If this step takes a while, don't be shy about producing `logging` outputs.

## Demand Response

In [10]:
raw_dr_eia861 = eia861_raw_dfs["demand_response_eia861"].copy()
tfr_dr_eia861 = eia_transformed_dfs["demand_response_eia861"].copy()

In [11]:
raw_dr_eia861.sample(10)

,balancing_authority_code_eia,commercial_actual_peak_demand_savings_mw,commercial_customer_incentives_cost,commercial_customers,commercial_energy_savings_mwh,commercial_other_costs,commercial_potential_peak_demand_savings_mw,industrial_actual_peak_demand_savings_mw,industrial_customer_incentives_cost,industrial_customers,industrial_energy_savings_mwh,industrial_other_costs,industrial_potential_peak_demand_savings_mw,report_year,residential_actual_peak_demand_savings_mw,residential_customer_incentives_cost,residential_customers,residential_energy_savings_mwh,residential_other_costs,residential_potential_peak_demand_savings_mw,state,total_actual_peak_demand_savings_mw,total_customer_incentives_cost,total_customers,total_energy_savings_mwh,total_other_costs,total_potential_peak_demand_savings_mw,transportation_actual_peak_demand_savings_mw,transportation_customer_incentives_cost,transportation_customers,transportation_energy_savings_mwh,transportation_other_costs,transportation_potential_peak_demand_savings_mw,utility_id_eia,utility_name_eia,water_heater
915,NaN,.,.,.,.,.,.,.,.,.,.,.,.,2014,.,.,.,.,.,.,MN,.,.,.,.,.,.,.,.,.,.,.,.,20806,City of Windom,.
1802,PJM,.,.,.,.,.,.,.,.,.,.,.,.,2017,0.4,0,35,1,1,0.5,MD,0.4,0,35,1,1,0.5,.,.,.,.,.,.,84,A & N Electric Coop,.
727,NaN,.,.,0,.,.,.,.,.,0,.,.,.,2014,.,.,0,.,.,.,OR,.,.,0,.,.,.,.,.,0,.,.,.,13788,Northern Wasco County PUD,.
2067,PJM,5.4,24,70,.,243,5.9,121.3,956,23,.,696,133.9,2017,.,.,.,.,.,.,PA,126.7,980,93,.,939,139.8,.,.,.,.,.,.,14715,PPL Electric Utilities Corp,.
1160,NaN,,,,,,,11.7,0,312,351,25,30,2015,,,,,,,GA,11.7,0,312,351,25,30,,,,,,,12472,Middle Georgia El Member Corp,0
1245,NaN,0,0,1,0,0,0.3,0,0,0,0,0,0,2015,0,0,0,0,0,0,NY,0,0,1,0,0,0.3,0,0,0,0,0,0,16183,Rochester Gas & Electric Corp,
1995,SWPP,0.5,.,20,.,1,0.5,0.5,.,2,.,1,0.5,2017,.,.,.,.,.,.,KS,1,.,22,.,2,1,.,.,.,.,.,.,12208,City of McPherson - (KS),.
161,NaN,47,2035,220,0,-300,47,0,0,0,0,0,0,2013,20,0,22251,0,508,20,MO,67,2035,22471,0,208,67,0,0,0,0,0,0,10000,Kansas City Power & Light Co,.
1506,NaN,.,.,.,.,.,.,.,.,.,.,.,.,2016,1.1,1,815,1,.,1.1,FL,1.1,1,815,1,.,1.1,.,.,.,.,.,.,7785,"Gulf Coast Electric Coop, Inc",.
365,NaN,0.3,0,153,0,0,1,0,0,0,0,0,0,2013,5.1,118,3275,0,27,8,ND,5.4,118,3428,0,27,9,.,.,.,.,.,.,19790,Verendrye Electric Coop Inc,.


In [12]:
tfr_dr_eia861.sample(10)

,utility_id_eia,state,balancing_authority_code_eia,utility_name_eia,water_heater,customer_class,actual_peak_demand_savings_mw,customer_incentives_cost,customers,energy_savings_mwh,other_costs,potential_peak_demand_savings_mw,report_date
5125,4045,MO,UNK,City of Columbia - (MO),0,commercial,NaN,47000.0,912,NaN,4000.0,4.0,2015-01-01
11109,40211,IL,MISO,"Wabash Valley Power Assn, Inc",<NA>,transportation,NaN,NaN,<NA>,NaN,NaN,NaN,2017-01-01
2174,807,AR,UNK,Arkansas Electric Coop Corp,<NA>,transportation,NaN,NaN,<NA>,NaN,NaN,NaN,2014-01-01
2437,3478,OK,UNK,Cimarron Electric Coop,<NA>,residential,NaN,NaN,<NA>,NaN,NaN,NaN,2014-01-01
6564,18820,MN,UNK,City of Thief River Falls,<NA>,transportation,NaN,NaN,<NA>,NaN,NaN,NaN,2015-01-01
10234,13780,WI,MISO,Northern States Power Co - Wisconsin,<NA>,transportation,NaN,NaN,<NA>,NaN,NaN,NaN,2017-01-01
7994,12796,WV,UNK,Monongahela Power Co,<NA>,transportation,NaN,NaN,<NA>,NaN,NaN,NaN,2016-01-01
5776,12268,TX,UNK,"Medina Electric Coop, Inc",<NA>,industrial,16.1,NaN,783,385.0,64000.0,78.5,2015-01-01
3110,10000,KS,UNK,Kansas City Power & Light Co,<NA>,commercial,0.0,0.0,670,0.0,2000.0,2.6,2014-01-01
41,407,GA,UNK,Altamaha Electric Member Corp,0,industrial,6.0,NaN,159,NaN,NaN,12.6,2013-01-01


In [13]:
tfr_dr_eia861.duplicated(subset=["report_date", "utility_id_eia", "state", "balancing_authority_code_eia", "customer_class"], keep=False).value_counts()

False    10580
dtype: int64

In [14]:
tfr_dr_eia861.customer_class.unique()

[commercial, industrial, residential, transportation]
Categories (4, object): [commercial, industrial, residential, transportation]

# Data Analysis and Visualization
* Now that you've got the required data in a usable form, you can tell the story of your analysis through a mix of visualizations, and further data wrangling steps.
* This narrative should be readable, with figures that have titles, legends, and labeled axes as appropriate so others can understand what you're showing them.
* The code should be concise and make use of the parameters and functions which you've defined above when possible. Functions should contain comprehensible chunks of work that make sense as one step in the story of the analysis.